In [1]:
import glob #to read the files
import json #to work if .json
import numpy as np #to math
import pandas as pd #to save the data
import math #to convertion calculus
from astropy.time import Time #to time converting
from astropy import units #time correction
from astropy.coordinates import SkyCoord #time correction
from scipy import interpolate #to interpolate the wavelength and flux
from tabulate import tabulate #to export in table format
from scipy.interpolate import interp1d
#import scipy.optimize as opt
from scipy import optimize
from scipy import signal
import os.path
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy import stats
import os.path 
import sncosmo

In [2]:
#kernel = 10.0 * RBF(length_scale=10.0, length_scale_bounds=(5, 15)) + WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e+1))

In [3]:
file = open("tn.txt")
lines = file.readlines()
file.close()

In [4]:
sn_names = []

for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    sn_names.append(co[0])

In [5]:
sn_names = ["SN2012fr"]

95

In [6]:
for nn in range(0, len(sn_names)):
    
    name = sn_names[nn]+".json"

    print(name)
    
    with open(name, "r") as read_file:
        data = json.load(read_file)

    url = name
    if url.endswith('.json'):
        url = url[:-5]

    %run ./functions.ipynb

    redshift = redshift_value(data[url])
    
    file = open("t0_values.txt")
    lines = file.readlines()
    file.close()

    t0_names = []
    t0_values = []

    for line in lines:
        if line.startswith('#'): continue
        co=line.rstrip().replace('INDEF','Nan').split()

        t0_names.append(co[0])
        t0_values.append(co[1])

    if url in t0_names:

        idx = t0_names.index(name[:-5])
        t0 = t0_values[idx]
        print("t0 calculado")

    else:

        t0 = max_time(data[url])

        print("t0 fornecido")

    #t0 = max_time(data[url])

    #max_time(data[url])

    #t0

    print(float(t0)-max_time(data[url]))

    units = []
    
    if redshift is not None and t0 is not None:

        #spectra data
        i=0
        n=0
        while i < len(data[url]["spectra"]):


            if "time" in data[url]["spectra"][i]:

                spectratime = float(data[url]["spectra"][i]["time"]) - float(t0)
                #print(spectratime)

            alert3 = 0
            #if its the first spectra then previous time is the first time
            if i == 0:
                previoustime = spectratime

            else:
                #if the diference between the time and previous time is more than * days so emmit an alert to continue
                if (spectratime - previoustime) < 2000:

                    alert3 = 1
                    previoustime = spectratime

                #if the diference of time is less than * continue
            if alert3 == 1:

                #verifing if its calibrated and if it have a redshift correction 
                alert1 = 0 #for calibration
                alert2 = 0 #for redshift correction

                if data[url]["spectra"][i]["u_fluxes"] == "erg/s/cm^2/Angstrom":          
                    alert1 = 1 #first alert

                    if "deredshifted" in data[url]["spectra"][i]:
                        alert2 = 1 #second alert

                #if its calibrated and corrected
                if alert1 == 1 and alert2 == 1:

                    #there are 3 collumns: wavelength flux and flux error 
                    if len(data[url]["spectra"][i]["data"][0]) == 3:

                        n = n + 1

                    #there are 2 collumns: wavelength and flux
                    elif len(data[url]["spectra"][i]["data"][0]) == 2:

                        n = n + 1

                #if its calibrated, redshift not adjusted but with a clear redshift
                elif alert1 ==1 and alert2 == 0:   

                    #there are 3 collumns: wavelength flux and flux error 
                    if len(data[url]["spectra"][i]["data"][0]) == 3:

                        n = n + 1

                    #there are 2 collumns: wavelength and flux
                    elif len(data[url]["spectra"][i]["data"][0]) == 2:

                        n = n + 1

            i = i + 1

    n

    final_gradex = np.linspace(-20, 50, 71)
    final_gradey = np.linspace(2000, 9200, 721)

    x = []
    y = []
    times = []

    if n > 4:

        #print(redshift)
        spectra_data = 0
        i=0

        while i < len(data[url]["spectra"]):

            if "time" in data[url]["spectra"][i]:

                spectratime = (float(data[url]["spectra"][i]["time"]) - float(t0))/(1+redshift)
                #print(spectratime)
                #verifing if its calibrated and if it have a redshift correction 
                alert1 = 0 #for calibration
                alert2 = 0 #for redshift correction

                if data[url]["spectra"][i]["u_fluxes"] == "erg/s/cm^2/Angstrom":          
                    alert1 = 1 #first alert

                    if "deredshifted" in data[url]["spectra"][i]:
                        alert2 = 1 #second alert

                #if its calibrated and corrected
                if alert1 == 1 and alert2 == 1:
                    #print("ok")
                    #there are 3 collumns: wavelength flux and flux error
                    if len(data[url]["spectra"][i]["data"][0]) == 3:

                        df = pd.DataFrame(data[url]["spectra"][i]["data"], columns=['wavelength', 'flux', 'fluxerror'])

                        #converting to float
                        df['wavelength'] = df['wavelength'].astype(float)
                        df['flux'] = df['flux'].astype(float)


                        #sort the data by wavelength
                        df = df.sort_values(by=['wavelength'])
                        df = df.reset_index(drop=True)

                        #to kill negative fluxes
                        fluxtemp = []
                        for var in range(0,len(df['flux'])):
                            if (df['flux'][var] > 0):
                                fluxtemp.append(df['flux'][var])
                            else:
                                fluxtemp.append(0)                                 
                                
                        fun = interp1d(df['wavelength'], fluxtemp, kind='cubic')        
                                
                        gridtemp = np.linspace(math.ceil(min(df['wavelength'])), math.ceil(max(df['wavelength']))-1, math.ceil(max(df['wavelength']))-1 - math.ceil(min(df['wavelength']))+1)
                        
                        #frequencia da amostra
                        fs = 1/(gridtemp[1]-gridtemp[0])
                                
                        interpflux = fun(gridtemp)        
                                

                        #making a list of lists 
                        suavf = suavization(interpflux, fs)

                        if "observer" in data[url]["spectra"][i]:

                            if data[url]["spectra"][i]["observer"] == "Zhang, Jujia":

                                x.append(gridtemp)
                                y.append(suavf)
                                times.append(spectratime)

                                units.append(data[url]["spectra"][i]["u_fluxes"]) 


                    #there are 2 collumns: wavelength and flux
                    if len(data[url]["spectra"][i]["data"][0]) == 2:

                        df = pd.DataFrame(data[url]["spectra"][i]["data"], columns=['wavelength', 'flux'])

                        #converting to float
                        df['wavelength'] = df['wavelength'].astype(float)
                        df['flux'] = df['flux'].astype(float)


                        #sort the data by wavelength
                        df = df.sort_values(by=['wavelength'])
                        df = df.reset_index(drop=True)

                        #to kill negative fluxes
                        fluxtemp = []
                        for var in range(0,len(df['flux'])):
                            if (df['flux'][var] > 0):
                                fluxtemp.append(df['flux'][var])
                            else:
                                fluxtemp.append(0)            

                                
                        fun = interp1d(df['wavelength'], fluxtemp, kind='cubic')        
                                
                        gridtemp = np.linspace(math.ceil(min(df['wavelength'])), math.ceil(max(df['wavelength']))-1, math.ceil(max(df['wavelength']))-1 - math.ceil(min(df['wavelength']))+1)
                        
                        #frequencia da amostra
                        fs = 1/(gridtemp[1]-gridtemp[0])
                                
                        interpflux = fun(gridtemp)        
                                

                        #making a list of lists 
                        suavf = suavization(interpflux, fs)

                        if "observer" in data[url]["spectra"][i]:

                            if data[url]["spectra"][i]["observer"] == "Zhang, Jujia":

                                x.append(gridtemp)
                                y.append(suavf)
                                times.append(spectratime)

                                units.append(data[url]["spectra"][i]["u_fluxes"]) 

                #if its calibrated, redshift not adjusted but with a clear redshift

                if alert1 ==1 and alert2 == 0:  

                    #there are 3 collumns: wavelength flux and flux error 
                    if len(data[url]["spectra"][i]["data"][0]) == 3:

                        df = pd.DataFrame(data[url]["spectra"][i]["data"], columns=['wavelength', 'flux', 'fluxerror'])

                        #converting to float
                        df['wavelength'] = df['wavelength'].astype(float)
                        df['flux'] = df['flux'].astype(float)


                        #sort the data by wavelength
                        df = df.sort_values(by=['wavelength'])
                        df = df.reset_index(drop=True)

                        #to kill negative fluxes
                        fluxtemp = []
                        for var in range(0,len(df['flux'])):
                            if (df['flux'][var] > 0):
                                fluxtemp.append(df['flux'][var])
                            else:
                                fluxtemp.append(0)


                        #redshift correction
                        fluxtemp = [x * (((1+redshift))**3) for x in fluxtemp] #for the flux           
                        df['wavelength'] = [x * (1/(1+redshift)) for x in df['wavelength']] #for the wavelength
                        #spectratime = spectratime/(1+redshift) #for time

                        #making a list of lists 
                                
                        fun = interp1d(df['wavelength'], fluxtemp, kind='cubic')        
                                
                        gridtemp = np.linspace(math.ceil(min(df['wavelength'])), math.ceil(max(df['wavelength']))-1, math.ceil(max(df['wavelength']))-1 - math.ceil(min(df['wavelength']))+1)
                        
                        #frequencia da amostra
                        fs = 1/(gridtemp[1]-gridtemp[0])
                                
                        interpflux = fun(gridtemp)        
                                

                        #making a list of lists 
                        suavf = suavization(interpflux, fs)

                        if "observer" in data[url]["spectra"][i]:

                            if data[url]["spectra"][i]["observer"] == "Zhang, Jujia":

                                x.append(gridtemp)
                                y.append(suavf)
                                times.append(spectratime)

                                units.append(data[url]["spectra"][i]["u_fluxes"]) 

                    #there are 2 collumns: wavelength flux and flux error 
                    if len(data[url]["spectra"][i]["data"][0]) == 2:

                        df = pd.DataFrame(data[url]["spectra"][i]["data"], columns=['wavelength', 'flux'])

                        #converting to float
                        df['wavelength'] = df['wavelength'].astype(float)
                        df['flux'] = df['flux'].astype(float)


                        #sort the data by wavelength
                        df = df.sort_values(by=['wavelength'])
                        df = df.reset_index(drop=True)

                        #to kill negative fluxes
                        fluxtemp = []
                        for var in range(0,len(df['flux'])):
                            if (df['flux'][var] > 0):
                                fluxtemp.append(df['flux'][var])
                            else:
                                fluxtemp.append(0)

                        #redshift correction
                        fluxtemp = [x * (((1+redshift))**3) for x in fluxtemp] #for the flux           
                        df['wavelength'] = [x * (1/(1+redshift)) for x in df['wavelength']] #for the wavelength
                        #spectratime = spectratime/(1+redshift) #for time

                                
                        fun = interp1d(df['wavelength'], fluxtemp, kind='cubic')        
                                
                        gridtemp = np.linspace(math.ceil(min(df['wavelength'])), math.ceil(max(df['wavelength']))-1, math.ceil(max(df['wavelength']))-1 - math.ceil(min(df['wavelength']))+1)
                        
                        #frequencia da amostra
                        fs = 1/(gridtemp[1]-gridtemp[0])
                                
                        interpflux = fun(gridtemp)        
                                

                        #making a list of lists 
                        suavf = suavization(interpflux, fs)


                        #if "instrument" in data[url]["spectra"][i]:

                        #if data[url]["spectra"][i]["instrument"] == "fast":

                        x.append(gridtemp)
                        y.append(suavf)
                        times.append(spectratime)

                        units.append(data[url]["spectra"][i]["u_fluxes"])  

            i = i + 1

    
    
    
    
    
    
    
    '''
    mag = []
    time = []
    system = []

    for i in range(0,len(data[url]["photometry"])):

        if "band" in data[url]["photometry"][i]:

            if data[url]["photometry"][i]["band"] == "V":

                if "system" in data[url]["photometry"][i]:

                    system.append(data[url]["photometry"][i]["system"])

                    mag.append(float(data[url]["photometry"][i]["magnitude"]))
                    time.append(float(data[url]["photometry"][i]["time"]))

                else:

                    system.append("w")

                    mag.append(float(data[url]["photometry"][i]["magnitude"]))
                    time.append(float(data[url]["photometry"][i]["time"]))

    
    csp = sncosmo.get_magsystem('csp')
    ab = sncosmo.get_magsystem('ab')
    vega = sncosmo.get_magsystem('vega')

    maxvalues = []

    if len(time) > 3:

        #maxvalues.append(min(mag))

        ntime = []
        nmag = []
        for i in range(0,len(mag)):

            if np.abs(time[i]- min(time)) < 100:

                if system[i] == "CSP":


                    nmag.append(csp.band_mag_to_flux(mag[i], 'cspv9844')*(1+redshift)**3)
                    ntime.append([time[i]])

                if system[i] == "AB":
                    #print("ab")

                    nmag.append(ab.band_mag_to_flux(mag[i], 'cspv9844')*(1+redshift)**3)
                    ntime.append([time[i]])

                if system[i] == "Vega":

                    #print("vega")
                    nmag.append(vega.band_mag_to_flux(mag[i], 'cspv9844')*(1+redshift)**3)
                    ntime.append([time[i]])   

                if system[i] == "w":

                    #print("w")
                    nmag.append(vega.band_mag_to_flux(mag[i], 'cspv9844')*(1+redshift)**3)
                    ntime.append([time[i]])                     

                #print(np.abs(time[i][0] - min(time[i])))                         

        if len(nmag) > 3:   

            gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(ntime, nmag)

            X_ = np.linspace(min(ntime)[0], max(ntime)[0], 1000)

            y_mean, y_cov = gp.predict(X_[:, np.newaxis], return_cov=True)

     
    len(x)

    wavenumber = 4000

    time = []
    flux = []

    for i in range(0,len(times)):

        if min(x[i]) < wavenumber < max(x[i]):

            interpolation = interp1d(x[i], y[i])

            time.append([times[i]])
            flux.append(interpolation(wavenumber))

   
    matplotlib notebook

    plt.figure(figsize=(8,6))
    plt.plot(time, flux,'k.')
    plt.xlabel('flux (erg/s/cm^2/Angstrom)')
    plt.ylabel('time (days)')
    plt.xlim(-20,50)
    plt.show()
    

    #time

    #max(y_mean)
    
    %matplotlib notebook
    plt.plot(X_, y_mean, 'k', lw=3, zorder=9)
    plt.plot(ntime, nmag,'ro')
    plt.fill_between(X_, y_mean - np.sqrt(np.diag(y_cov)),
                     y_mean + np.sqrt(np.diag(y_cov)),
                     alpha=0.5, color='k')
    plt.show()
    
    
    '''
    
    
    
    
    


    save_path = '/home/joao/Documentos/templates/data'

    for i in range(0,len(x)):

        table = []
        for k in range(0,len(x[i])):
            table.append((x[i][k], y[i][k]))


        name_of_file = str(url)

        completeName = os.path.join(save_path, name_of_file+str(i)+".dat")      

        header = '#' + str(times[i])

        f = open(completeName, 'w')               
        f.write(header + "\n")
        f.write(tabulate(table, tablefmt="plain"))
        f.close()



        #print(i)


SN2005cf.json
0.006461
t0 calculado
0.4000000000014552
SN2006X.json
0.00524
t0 calculado
0.19999999999708962
SN2001V.json
0.01502
t0 calculado
-0.19999999999708962
SN2003du.json
0.006381
t0 calculado
0.5
SN2011fe.json
0.000804
t0 calculado
4.69999999999709
SN2014J.json
0.000841
t0 calculado
2.099999999998545
SN1994D.json
0.00149
t0 calculado
0.4000000000014552
SN2004dt.json
0.01973
t0 calculado
0.5999999999985448
SN2002bo.json
0.00424
t0 calculado
-1.0999999999985448
SN1998aq.json
0.003699
t0 calculado
0.6999999999970896
SN2007le.json
0.006721
t0 calculado
-1.4000000000014552
SN2007af.json
0.005464
t0 calculado
1.0
SN1999aa.json
0.01444
t0 calculado
0.9000000000014552
SN2000cx.json
0.007929
t0 calculado
0.0
SN2009dc.json
0.02158
t0 calculado
0.3000000000029104
SN1999ac.json
0.0115
t0 calculado
0.19999999999708962
SN1999by.json
0.002128
t0 calculado
0.19999999999708962
SN2008ar.json
0.026147
t0 calculado
0.4000000000014552
SN2001ep.json
0.013012
t0 calculado
6.5
SN2005am.json
0.007899
t